In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing
import time
import random
import logging

from sklearn.model_selection import train_test_split
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn import metrics as skm
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

logging.basicConfig(
    level=logging.INFO,
    #format="%(asctime)s %(message)s",
    format="%(message)s",
    handlers=[
        logging.FileHandler("{0}.log".format('pitch_prediction')),
        logging.StreamHandler()
    ])

logger = logging.getLogger()

pd.options.display.width = 0
pd.set_option('display.max_rows', None)

Using TensorFlow backend.


In [ ]:
# decode labels for outcomes and pitch types

# outcome codes
outcome_code = {
    'outcome_code' : ['B', '*B', 'S', 'C',
                      'F', 'T', 'L', 'I',
                      'W', 'M', 'P',
                      'Q','R', 'X',
                      'D', 'E', 'H',
                      'V', 'Z'],
    'outcome_description' : ['Ball', 'Ball in Dirt', 'Swinging Strike', 'Called Strike',
                            'Foul', 'Foul Tip', 'Foul Bunt', 'Intentional Ball',
                            'Swinging Strike (Blocked)', 'Missed Bunt', 'Pitchout',
                            'Swinging Pitchout', 'Foul Pitchout', 'In Play, Out(s)',
                            'In Play, No Outs', 'In Play, Runs', 'Hit by pitch',
                            'V', 'Z']
}
outcome_code = pd.DataFrame.from_dict(outcome_code)

pitch_code = {
    'pitch_code': ['CH', 'CU', 'EP', 'FC',
                   'FF', 'FO', 'FS', 'FT',
                   'IN', 'KC', 'KN',
                   'PO', 'SC', 'SI',
                   'SL', 'UN', 'FA', 'AB'],
    'pitch_description': ['Changeup', 'Curveball', 'Eephus', 'Cutter',
                          'Four-seam Fastball', 'Pitchout', 'Splitter', 'Two-seam Fastball',
                          'Intentionall ball', 'Knuckle curve', 'Knuckleball',
                          'Pitchout', 'Screwball', 'Sinker',
                          'Slider', 'Unknown', 'FA', 'AB']
}
pitch_code = pd.DataFrame.from_dict(pitch_code)

In [ ]:
def load_data(num_rec=0):
    # specify subset of records to use
    if num_rec > 0:
        pitches = pd.read_csv("data/pitches.csv", nrows=num_rec)
    else:
        pitches = pd.read_csv("data/pitches.csv")
    atbats = pd.read_csv("data/atbats.csv")
    ejections = pd.read_csv("data/ejections.csv")
    games = pd.read_csv("data/games.csv")
    player_names = pd.read_csv("data/player_names.csv")



    # join all the data into all_df
    all_df = pd.merge(pitches, atbats, how='left',
            left_on = 'ab_id', right_on = 'ab_id')

    pitcher_df = player_names
    pitcher_df.columns = ['pitcher_id', 'pitcher_first_name', 'pitcher_last_name']

    all_df = pd.merge(all_df, pitcher_df, how='left',
            left_on = 'pitcher_id', right_on = 'pitcher_id')

    batter_df = player_names
    batter_df.columns = ['batter_id', 'batter_first_name', 'batter_last_name']

    all_df = pd.merge(all_df, batter_df, how='left',
            left_on = 'batter_id', right_on = 'batter_id')

    all_df = pd.merge(all_df, games, how='left',
            left_on = 'g_id', right_on = 'g_id')

    all_df = pd.merge(all_df, pitch_code, how='left',
            left_on = 'pitch_type', right_on = 'pitch_code')

    all_df = pd.merge(all_df, outcome_code, how='left',
            left_on = 'code', right_on = 'outcome_code')

    # extract weather conditions
    all_df['temp'] = all_df['weather'].str.extract(r'(\d+)')  # temperature
    all_df['temp'] = pd.to_numeric(all_df['temp'])
    all_df['weather_cond'] = all_df['weather'].str.extract(r'([^,]*$)')  # weather condition
    all_df['wind_mph'] = all_df['wind'].str.extract(r'(\d+)')  # wind speed
    all_df['wind_mph'] = pd.to_numeric(all_df['wind_mph'])
    all_df['wind_dir'] = all_df['wind'].str.extract(r'([^,]*$)')  # wind direction

    all_df.dropna(inplace=True)


    return all_df

def encode_data(df_in, x_col_list, y_col, avail_cols, y_encode='none', factor=np.nan, x_encode=True):
    col_no_encode = []
    # we know there's one dependent variable to seed a dataframe
    y_df = df_in[[y_col]]
    encode_df = y_df

    for col in x_col_list:
        if avail_cols.get(col):
            # one-hot encode
            dummies_df = pd.get_dummies(df_in[col], prefix=col)
            encode_df = pd.concat([encode_df, dummies_df], axis=1)
        else:
            # just retrieve the column directly
            col_no_encode.append(col)

    encode_df = encode_df.drop([y_col], axis=1) # remove dependent variable seeding column
    if x_encode:
        X = pd.concat([encode_df, df_in[col_no_encode]], axis=1)
    else:
        X = df_in[x_col_list]

    if y_encode=='factorize':
        # use factorized y value
        y_return = factor[0]
    elif y_encode=='onehot':
        # one hot encode y values
        y_return = pd.get_dummies(y_df, prefix=y_col)
    else:
        y_return = y_df

    y = y_return

    return X, y


In [ ]:
# functions to evaluate the results of different models and write to file

def format_cm(y_test_in, y_pred_in, factor_in):
    # confusion matrix
    y_test_label = factor_in[1][y_test_in]
    y_pred_label = factor_in[1][y_pred_in]
    cm = pd.DataFrame(
        confusion_matrix(y_test_label, y_pred_label, labels=list(factor_in[1])),
        index=list(factor_in[1]),
        columns=list(factor_in[1])
    )
    return cm


def eval_rf(y_test_in, y_pred_in, classifier_in, factor_in, idx_in, vars_in, elapsed_time, col_labels):
    logger.info(f'\nRandom Forest Test ' + str(idx_in))
    logger.info(str(vars_in))

    # confusion matrix
    cm = format_cm(y_test_in, y_pred_in, factor_in)
    logger.info(cm)

    # print variable importance
    #df_x_train = pd.DataFrame(X_train)

    feature_importances = pd.DataFrame(classifier_in.feature_importances_,
                                       index = col_labels,
                                        columns=['importance']).sort_values('importance', ascending=False)
    #train_col_df = pd.DataFrame(df_x_train.columns)
    #fi = pd.merge(feature_importances, col_labels_df, how='left',
    #        left_index = True, right_index = True)

    #logger.info('\n' + str(feature_importances.iloc[:20]))
    logger.info('\n' + str(feature_importances))

    pscore = metrics.accuracy_score(y_test_in, y_pred_in)
    logger.info(f'Random Forest accuracy {pscore}')
    logger.info('Model time: %.1f [sec]' % (elapsed_time))

def eval_knn(y_test_in, y_pred_in, factor_in, idx_in, vars_in, elapsed_time):
    logger.info(f'\nKNN Test ' + str(idx_in))
    logger.info(str(vars_in))

    # confusion matrix
    cm = format_cm(y_test_in, y_pred_in, factor_in)
    logger.info(cm)

    # accuracy/time
    pscore = metrics.accuracy_score(y_test_in, y_pred_in)
    logger.info(f'KNN accuracy {pscore}')
    logger.info('Model time: %.1f [sec]' % (elapsed_time))

def eval_svc(y_test_in, y_pred_in, factor_in, idx_in, vars_in, elapsed_time):
    logger.info(f'\nSVC Test ' + str(idx_in))
    logger.info(str(vars_in))

    # confusion matrix
    cm = format_cm(y_test_in, y_pred_in, factor_in)
    logger.info(cm)

    # accuracy/time
    pscore = metrics.accuracy_score(y_test_in, y_pred_in)
    logger.info(f'SVC accuracy {pscore}')
    logger.info('Model time: %.1f [sec]' % (elapsed_time))

def eval_nn(y_test_in, y_pred_in, history_in, factor_in, idx_in, vars_in, elapsed_time_in):
    logger.info(f'\nNeural Net Test ' + str(idx_in))
    logger.info(str(vars_in))

    # model architecture
    print(str(history_in.model.summary()))
    logger.info(str(history_in.model.summary()))

    # accuracy/loss by epoch
    hist = pd.DataFrame(history_in.history)
    hist['epoch'] = history_in.epoch
    logger.info('\n' + str(hist))

    # plot
    acc = history_in.history['accuracy']
    val_acc = history_in.history['val_accuracy']
    loss = history_in.history['loss']
    val_loss = history_in.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # exit accuracy and loss
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    logger.info(f'\nExit test accuracy: {test_acc}')
    logger.info(f'Exit test loss: {test_loss}')

    # confusion matrix
    cm = format_cm(y_test_in, y_pred_in, factor_in)
    logger.info(cm)

    # accuracy/time
    pscore = metrics.accuracy_score(y_test_in, y_pred_in)
    logger.info(f'Neural Net accuracy {pscore}')
    logger.info('Model time: %.1f [sec]' % (elapsed_time_in))

In [ ]:
# These are the values one can pass in as variables
# Values specify whether the column should be one-hot encoded
# Some are known at the time of pitch and some are outcomes of a pitch
avail_cols = {
    'px': False,        'pz': False,        'start_speed': False,   'end_speed': False,
    'spin_rate': False, 'spin_dir': False,  'break_angle': False,   'break_length': False,
    'break_y': False,   'ax': False,        'ay': False,            'az': False,
    'sz_bot': False,    'sz_top': False,    'type_confidence': True,'vx0': False,
    'vy0': False,       'vz0': False,       'x': False,             'x0': False,
    'y': False,         'y0': False,        'z0': False,            'pfx_x': False,
    'pfx_z': False,     'nasty': False,     'zone': True,           'code':	True,
    'type': True,       'pitch_type': True, 'event_num': False,     'b_score': False,
    'ab_id' : True,     'b_count': True,    's_count': True,        'outs': True,
    'pitch_num': False, 'on_1b': False,     'on_2b': False,         'on_3b': False,
    'batter_id': True,  'event': True,      'g_id': True,           'inning': True,     'o': True,
    'p_score': False,   'p_throws': True,   'pitcher_id': True,     'stand': True,      'top': False,
    'pitcher_first_name': True,     'pitcher_last_name': True,      'batter_first_name': True,
    'batter_last_name': True,       'attendance': False,            'away_final_score': False,
    'away_team': True,  'date': True,       'elapsed_time': False,  'home_final_score': False,
    'home_team': True,  'start_time': False, 'umpire_1B': True,      'umpire_2B': True,
    'umpire_3B': True,  'umpire_HP': True,  'venue_name': True,     'weather': True,
    'wind': True,       'delay': False,     'pitch_code': True,     'pitch_description': True,
    'outcome_code': True,           'outcome_description': True,    'temp': False,
    'weather_cond': True,           'wind_mph': False,              'wind_dir': True
}

Load the data from the files 

In [ ]:
# load data, calculate new columns, join data - optional number of records parameter
all_df = load_data(10000) # pick the first n records if desired
#all_df = load_data()
#all_df = all_df[(all_df['date'] >= '2018-01-01') & (all_df['date'] <= '2018-12-31')]  # 2018

Data for baseline statistics

In [ ]:
# pitch outcomes
outcome_df = all_df.groupby(['outcome_code', 'outcome_description']).size().reset_index(name='outcome_count')
outcome_df['outcome_prob']= outcome_df['outcome_count']/outcome_df['outcome_count'].sum()
outcome_df

In [ ]:
# bar chart of pitch outcomes
objects = outcome_df.outcome_description
y_pos = np.arange(len(outcome_df.outcome_description))
performance = outcome_df.outcome_count

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Pitches')
plt.title('MLB 2015-2018')

plt.show()

In [ ]:
# pitch outcomes
pitch_df = all_df.groupby(['pitch_code', 'pitch_description']).size().reset_index(name='pitch_type_count')

pitch_df['pitch_prob']= pitch_df['pitch_type_count']/pitch_df['pitch_type_count'].sum()
pitch_df

In [ ]:
# bar chart of pitch types
objects = pitch_df.pitch_description
y_pos = np.arange(len(pitch_df.pitch_description))
performance = pitch_df.pitch_type_count

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Pitches')
plt.title('MLB 2015-2018')

plt.show()

In [ ]:
# create one list of variables for each test you want to run
var_list = []

# this list has all the predictors known before a pitch takes place
# var_list.append(['b_score', 'b_count', 's_count','pitch_num', 'on_1b', 'on_2b', 'on_3b', 'batter_id', 'inning', 'o',
#                  'p_score', 'p_throws', 'stand', 'top', 'batter_id', 'pitcher_id',
#                  'away_team', 'home_team', 'umpire_HP','venue_name', 'temp', 'weather_cond',
#                  'wind_mph', 'wind_dir', 'delay'])

# this section for passing in fixed lists of predictors
# This list has the best prediction so far
var_list.append(['top', 'on_1b', 'on_2b', 'on_3b', 'o', 'p_score', 'stand', 'delay', 'b_count', 's_count',
               'pitcher_id', 'pitch_num'])
# var_list.append(['pitch_num', 'top', 'on_1b', 'on_2b', 'on_3b', 'o', 'p_score', 'stand', 'delay', 'b_count', 's_count', 'pitcher_id'])
fixed_vars = []

# this section for mixing a fixed number of predictors with a set number of predictors
# put the list of fixed predictors in the fixed_vars list
# put the mutually exclusive list of candidate predictors is the var_options list

# var_options = []
# fixed_vars  =['top', 'on_1b', 'on_2b', 'on_3b', 'o', 'p_score', 'stand', 'delay', 'b_count', 's_count',
#               'pitcher_id', 'pitch_num']
# var_options.append(['b_score', 'batter_id', 'inning', 'p_throws',
#                  'away_team', 'home_team', 'umpire_HP','venue_name', 'temp',
#                  'weather_cond', 'wind_mph', 'wind_dir'])
# var_list = list(combinations(var_options[0], 1))
# random.shuffle(var_list)


# factorize dependent variable
factor = pd.factorize(all_df['pitch_code'])
pitch_code_def = factor[1]


Linear Model (fails to converge)

In [ ]:
# loop through different combinations of predictors and run linear model
# NOTE: the linear model doesn't converge
# for idx, vars in enumerate(var_list):
#     var1 = list(vars)
#     var1.extend(fixed_vars)
#     vars = var1
#
#     # one-hot encode all necessary data and split into test/training data
#     X, y = encode_data(all_df, x_col_list=vars, y_col='pitch_code', avail_cols=avail_cols,
#                        y_encode='factorize', factor=factor)
#
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
#
#     X_train = sklearn.preprocessing.scale(X_train, copy=False)
#     X_test = sklearn.preprocessing.scale(X_test, copy=False)
#
#     mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='saga').fit(X_train, y_train)

Start of Random Forest calculations

In [ ]:
#loop through different combinations of predictors and run random forest calculations
for idx, vars in enumerate(var_list):

    # one-hot encode all necessary data and split into test/training data
    X, y = encode_data(all_df, x_col_list=vars, y_col='pitch_code', avail_cols=avail_cols,
                       y_encode='factorize', factor=factor)
    #X = sklearn.preprocessing.scale(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
    X_train = sklearn.preprocessing.scale(X_train, copy=False)
    X_test = sklearn.preprocessing.scale(X_test, copy=False)

    t1_start = time.perf_counter()

    # create and train random forest model
    classifier = RandomForestClassifier(n_estimators = 50, criterion = 'entropy', random_state = 42)
    classifier.fit(X_train, y_train)

    # predict test data
    y_pred = classifier.predict(X_test)

    t1_stop = time.perf_counter()
    elapsed_time = t1_stop - t1_start

    # print metrics for random forest model
    eval_rf(y_test, y_pred, classifier, factor, idx, vars, elapsed_time, X.columns)

K-Nearest Neighbor

In [ ]:
#loop through different combinations of predictors and run k-nearest neighbor
for idx, vars in enumerate(var_list):
    # add the fixed vars to the random ones
    var1 = list(vars)
    var1.extend(fixed_vars)
    vars = var1

    # one-hot encode all necessary data and split into test/training data
    X, y = encode_data(all_df, x_col_list=vars, y_col='pitch_code', avail_cols=avail_cols,
                       y_encode='factorize', factor=factor)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

    X_train = sklearn.preprocessing.scale(X_train, copy=False)
    X_test = sklearn.preprocessing.scale(X_test, copy=False)

    t1_start = time.perf_counter()

    knn = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
    knn.fit(X_train, y_train)

    y_pred = knn.predict(X_test)

    t1_stop = time.perf_counter()
    elapsed_time = t1_stop - t1_start
    eval_knn(y_test, y_pred, factor, idx, vars, elapsed_time)

SVC

In [ ]:
#loop through different combinations of predictors and run SVM calculations
for idx, vars in enumerate(var_list):

    # one-hot encode all necessary data and split into test/training data
    X, y = encode_data(all_df, x_col_list=vars, y_col='pitch_code', avail_cols=avail_cols,
                       y_encode='factorize', factor=factor)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)
    X_train = sklearn.preprocessing.scale(X_train, copy=False)
    X_test = sklearn.preprocessing.scale(X_test, copy=False)

    # tuned the SVC model on a small (10K) number of records using grid search
    #params_grid = [{'kernel': ['rbf'], 'gamma': [0.1, 1, 10, 100], 'C': [0.1, 1, 10, 100, 1000]},
    #               {'kernel': ['poly'], 'degree': [1,2,3,4,5,6], 'C': [0.1, 1, 10, 100, 1000]}
    #               # ,{'kernel': ['linear'], 'C': [1, 10, 100, 1000]} # linear classifier not suitable for this data
    #               ]
    #svm_model = GridSearchCV(SVC(gamma='auto'), params_grid, cv=2)
    #print('Best score for training data:', svm_model.best_score_, "\n")
    #print('Best C:', svm_model.best_estimator_.C, "\n")
    #print('Best Kernel:', svm_model.best_estimator_.kernel, "\n")
    #print('Best Gamma:', svm_model.best_estimator_.gamma, "\n")
    #final_model = svm_model.best_estimator_

    t1_start = time.perf_counter()

    # used grid search to find best parameters
    final_model= SVC(C=1, kernel='poly', degree=1)
    final_model.fit(X_train, y_train)

    y_pred = final_model.predict(X_test)

    t1_stop = time.perf_counter()
    elapsed_time = t1_stop - t1_start

    eval_svc(y_test, y_pred, factor, idx, vars, elapsed_time)

Start of Multi-layer network (MLP)

In [ ]:
# loop through different combinations of predictors and run neural network calculations
for idx, vars in enumerate(var_list):
    # add the fixed vars to the random ones
    var1 = list(vars)
    var1.extend(fixed_vars)
    vars = var1

    # one-hot encode all necessary data and split into test/training data
    X, y = encode_data(all_df, x_col_list=vars, y_col='pitch_code', avail_cols=avail_cols,
                       y_encode='onehot', factor=factor)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

    X_train = sklearn.preprocessing.scale(X_train, copy=False)
    X_test = sklearn.preprocessing.scale(X_test, copy=False)

    t1_start = time.perf_counter()

    model = Sequential()
    model.add(Dense(512, input_dim=np.size(X_train,1), activation='relu'))
    model.add(Dense(256, input_dim=np.size(X_train,1), activation='relu'))
    model.add(Dense(np.size(y_train,1), activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    history = model.fit(X_train,
                        y_train,
                        epochs=10,
                        batch_size=512,
                        validation_data=(X_test, y_test),
                        verbose=True)

    y_pred = model.predict(X_test)

    #matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

    t1_stop = time.perf_counter()
    elapsed_time = t1_stop - t1_start

    factor = pd.factorize([x.split('pitch_code_')[1] for x in y_test.columns])
    eval_nn(np.array(y_test).argmax(axis=1), y_pred.argmax(axis=1), history, factor, idx, vars, elapsed_time)

In [ ]:
# experimenting with FastAI
from fastai.tabular import *

for idx, vars in enumerate(var_list):
    # have to add the dependent variable back in
    fastai_var_list = vars
    fastai_var_list.append('pitch_code')
    df = all_df[fastai_var_list]
    procs = [FillMissing, Categorify, Normalize]
    dep_var = 'pitch_code'
    len = len(df)
    valid_idx = range(len-(int(len*.2)), len)
    cat_names = ['o', 'stand', 'b_count', 's_count', 'pitcher_id']
    procs = [FillMissing, Categorify, Normalize]

    data = TabularDataBunch.from_df('.', df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
    print(data.train_ds.cont_names)  # `cont_names` defaults to: set(df)-set(cat_names)-{dep_var}

    logger.info('FastAI Model')
    t1_start = time.perf_counter()

    learn = tabular_learner(data, layers=[512,256], emb_szs={'native-country': 10}, metrics=accuracy)
    learn.fit_one_cycle(8, 1e-2)
    t1_stop = time.perf_counter()
    elapsed_time = t1_stop - t1_start

    logger.info('Model time: %.1f [sec]' % (elapsed_time))
